In [1]:
import pandas as pd
import numpy as np

from datasets import load_dataset

In [2]:
EOS_TOKEN = "<|endoftext|>"

In [3]:
dd = load_dataset("daily_dialog")
dd

/home/admin/miniconda3/envs/innprac37/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
Using custom data configuration default
Reusing dataset daily_dialog (/home/admin/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 11118
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
})

In [4]:
def show_dialog(item):
  chatters = ['A', 'B']
  i = 0
  for j, utt in enumerate(item['dialog']):
    print(f"{chatters[i]}: {utt} [{item['act'][j]} {item['emotion'][j]}]")
    i += 1
    i %= 2

In [5]:
show_dialog(next(iter(dd["train"])))

A: Say , Jim , how about going for a few beers after dinner ?  [3 0]
B:  You know that is tempting but is really not good for our fitness .  [4 0]
A:  What do you mean ? It will help us to relax .  [2 0]
B:  Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ?  [2 0]
A:  I guess you are right.But what shall we do ? I don't feel like sitting at home .  [2 0]
B:  I suggest a walk over to the gym where we can play singsong and meet some of our friends .  [3 0]
A:  That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them .  [4 4]
B:  Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too .  [1 4]
A:  Good.Let ' s go now .  [3 4]
B:  All right .  [4 4]


In [8]:
def add_prompt(batch):
  dialogs = batch["dialog"]
  acts = batch["act"]
  for i in range(len(dialogs)):
    for j in range(len(dialogs[i])):
      batch["dialog"][i][j] = "".join([f"[DA_{acts[i][j]}] ", dialogs[i][j], EOS_TOKEN])
  return batch

In [9]:
prompted_dd = dd.map(add_prompt, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
show_dialog(next(iter(prompted_dd["train"])))

A: [DA_3] Say , Jim , how about going for a few beers after dinner ? <|endoftext|> [3 0]
B: [DA_4]  You know that is tempting but is really not good for our fitness . <|endoftext|> [4 0]
A: [DA_2]  What do you mean ? It will help us to relax . <|endoftext|> [2 0]
B: [DA_2]  Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? <|endoftext|> [2 0]
A: [DA_2]  I guess you are right.But what shall we do ? I don't feel like sitting at home . <|endoftext|> [2 0]
B: [DA_3]  I suggest a walk over to the gym where we can play singsong and meet some of our friends . <|endoftext|> [3 0]
A: [DA_4]  That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . <|endoftext|> [4 4]
B: [DA_1]  Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . <|endoftext|> [1 4]
A: [DA_3]  Good.Let ' s go now . <|endoftext|> [3 4]
B: [DA_4]  All 

In [29]:
train = {
  "history": [],
  "response": [],
  "da": []
}

test = {
    "history": [],
    "response": [],
    "da": []
}

val = {
  "history": [],
  "response": [],
  "da": []
}

In [30]:
def make_history(dialog):
  assert len(dialog) >= 2
  history = [None] * (len(dialog) - 1)
  response = [None] * (len(dialog) - 1)
  history[0] = dialog[0]
  response[0] = dialog[1]
  for j in range(1, len(dialog) - 1):
    history[j] = "".join([history[j - 1], response[j - 1]])
    response[j] = dialog[j + 1]
  return history, response
  

def dataset_to_dict(src):
  dest = {
    "history": [],
    "response": [],
    "da": []
  }
  for item in src:
    history, response = make_history(item["dialog"])
    dest["history"].extend(history)
    dest["response"].extend(response)
    dest["da"].extend(item["act"][1:])
  return dest

In [33]:
train = dataset_to_dict(prompted_dd["train"])
test = dataset_to_dict(prompted_dd["test"])
val = dataset_to_dict(prompted_dd["validation"])
(len(train["da"]), len(test["da"]), len(val["da"]))

(76052, 6740, 7069)

In [42]:
import os

def save_dataset(data: dict, path):
  print(f"Saving to {path}")
  with open(path, "w") as f:
    df = pd.DataFrame(data)
    df.to_csv(f, index=False, sep=";")
  print("Saved")

In [43]:
save_dataset(train, "../data/da_prompts/train.csv")
save_dataset(test, "../data/da_prompts/test.csv")
save_dataset(val, "../data/da_prompts/val.csv")

Saving to ../data/da_prompts/train.csv
Saved
Saving to ../data/da_prompts/test.csv
Saved
Saving to ../data/da_prompts/val.csv
Saved
